In [1]:
%cd ..
%load_ext nb_black

/home/hardianlawi/attention-lstm-sentiment-analysis


<IPython.core.display.Javascript object>

In [2]:
import json
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from src.preprocess import Preprocessor
from src.data import get_data
from src.models import get_model

tf.random.set_seed(2020)

vocab_size = 5000
emb_size = 32
max_words = 500
oov_token = "<OOV>"

(str_X_train, y_train), (str_X_val, y_val), (str_X_test, y_test) = get_data()

preprocessor = Preprocessor(
    maxlen=max_words, vocab_size=vocab_size, oov_token=oov_token
)
preprocessor.fit_on_texts(str_X_train + str_X_val + str_X_test)

X_train = preprocessor.transform(str_X_train)
X_val = preprocessor.transform(str_X_val)
X_test = preprocessor.transform(str_X_test)

<IPython.core.display.Javascript object>

In [3]:
def generate_scores(i):

    models = []
    num_params = []
    accs = []

    def train(hs):

        model = get_model("lstm", max_words, vocab_size, emb_size, hidden_size=hs,)
        model.compile(
            loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        batch_size = 128
        num_epochs = 3

        model.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            batch_size=batch_size,
            epochs=num_epochs,
        )

        scores = model.evaluate(X_test, y_test, verbose=0)
        print("Test accuracy:", scores[1])

        return scores[1], model.count_params(), model

    hparams = [32, 64]
    for hs in hparams:

        print(f"{(hs)}")

        acc, num_param, model = train(hs)
        accs.append(float(acc))
        num_params.append(num_param)
        models.append(model)

    perfs = {
        "hparams": hparams,
        "num_params": [int(x) for x in num_params],
        "accs": [float(x) for x in accs],
    }

    with open(f"lstm_perfs_{i}.json", "w") as f:
        json.dump(perfs, f)


for i in range(3):
    generate_scores(i)

32
Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 89s 4ms/sample - loss: 0.4668 - accuracy: 0.7777 - val_loss: 0.3288 - val_accuracy: 0.8592
Epoch 2/3
23750/23750 [==============================] - 83s 4ms/sample - loss: 0.2704 - accuracy: 0.8947 - val_loss: 0.2783 - val_accuracy: 0.8824
Epoch 3/3
23750/23750 [==============================] - 83s 3ms/sample - loss: 0.2400 - accuracy: 0.9080 - val_loss: 0.3259 - val_accuracy: 0.8808
Test accuracy: 0.86988
64
Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================] - 109s 5ms/sample - loss: 0.4514 - accuracy: 0.7875 - val_loss: 0.2945 - val_accuracy: 0.8776
Epoch 2/3
23750/23750 [==============================] - 104s 4ms/sample - loss: 0.2704 - accuracy: 0.8917 - val_loss: 0.2748 - val_accuracy: 0.8856
Epoch 3/3
23750/23750 [==============================] - 106s 4ms/sample - loss: 0.2275 - accuracy: 0.9141 - val_loss: 0.2716 - val

<IPython.core.display.Javascript object>